In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import torch as t

from master_thesis.core.utils.reproducibility import seed_everything, load_checkpoint
from master_thesis.core.models.llama import load_model_and_tokenizer
from master_thesis.core.utils.prompts import load_prompt
from master_thesis.core.activations.collector import ActivationCollector

seed_everything()

In [2]:
MODEL = "LLAMA_2_7B_CHAT"
DATA_DIR = f"../../../../data"
DATASETS_DIR = f"{DATA_DIR}/datasets/base_experiments/europe_vs_usa"
ACTIVATIONS_DIR = f"{DATA_DIR}/activations/{MODEL}/base_experiments/europe_vs_usa"
PROMPT_TYPE = "few_shot"
DEVICE = "cuda"

In [3]:
model, tokenizer = load_model_and_tokenizer(MODEL)

/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:732: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/mateusz_politycki/master-thesis-probing/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
layers = list(range(len(model.model.layers)))

In [5]:
label_prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/europe_vs_usa",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="label_aspect",
)

confounding_prompt = load_prompt(
    DATA_DIR,
    dataset_path="base_experiments/europe_vs_usa",
    prompt_type=PROMPT_TYPE,
    prompt_aspect="confounding_aspect",
)

In [ ]:
activation_collector = ActivationCollector(model, tokenizer, layers, DEVICE)

### Test activations

In [12]:
dataset = pd.read_csv(f"{DATASETS_DIR}/test/test.csv")
statements = dataset["sentence"].tolist()


for prompt_name, prompt in {
    "label_prompt": label_prompt,
    "confounding_prompt": confounding_prompt,
}.items():
    save_dir = f"{ACTIVATIONS_DIR}/test/{PROMPT_TYPE}_{prompt_name}"
    Path(save_dir).mkdir(parents=True, exist_ok=True)

    checkpoint = load_checkpoint(save_dir)

    for idx in tqdm(range(checkpoint, len(statements), 25)):
        acts, answers = activation_collector.get_acts(
            statements[idx : idx + 25], prompt
        )
        for layer, act in acts.items():
            t.save(act, f"{save_dir}/layer_{layer}_{idx}.pt")
        answers_df = pd.DataFrame({"answer": answers})
        answers_df.to_csv(f"{save_dir}/answers_{idx}.csv")

100%|██████████| 16/16 [02:54<00:00, 10.91s/it]


### Train - label_aspect

In [14]:
dataset = pd.read_csv(f"{DATASETS_DIR}/train/label_aspect.csv")
statements = dataset["sentence"].tolist()


save_dir = f"{ACTIVATIONS_DIR}/train/{PROMPT_TYPE}_label_aspect"
Path(save_dir).mkdir(parents=True, exist_ok=True)

checkpoint = load_checkpoint(save_dir)

for idx in tqdm(range(checkpoint, len(statements), 25)):
    acts, answers = activation_collector.get_acts(
        statements[idx : idx + 25], label_prompt
    )
    for layer, act in acts.items():
        t.save(act, f"{save_dir}/layer_{layer}_{idx}.pt")
    answers_df = pd.DataFrame({"answer": answers})
    answers_df.to_csv(f"{save_dir}/answers_{idx}.csv")

100%|██████████| 8/8 [01:24<00:00, 10.62s/it]


### Train - confounding aspect

In [15]:
dataset = pd.read_csv(f"{DATASETS_DIR}/train/confounding_aspect.csv")
statements = dataset["sentence"].tolist()


save_dir = f"{ACTIVATIONS_DIR}/train/{PROMPT_TYPE}_confounding_aspect"
Path(save_dir).mkdir(parents=True, exist_ok=True)

checkpoint = load_checkpoint(save_dir)

for idx in tqdm(range(checkpoint, len(statements), 25)):
    acts, answers = activation_collector.get_acts(
        statements[idx : idx + 25], confounding_prompt
    )
    for layer, act in acts.items():
        t.save(act, f"{save_dir}/layer_{layer}_{idx}.pt")
    answers_df = pd.DataFrame({"answer": answers})
    answers_df.to_csv(f"{save_dir}/answers_{idx}.csv")

100%|██████████| 8/8 [01:29<00:00, 11.13s/it]
